In [ ]:
'''
Visualization - not very informative ; Preprocessing --> dont touch rank, results becomes even worse 
Feature engineering - not much to be done, more complex features worsen results :(
Model selection - naive bayes=junk, rf - soso, xgboost - at least something , nn...
Hyperparameters tuning -- too much time
Evaluation(metrics) - get probs from classifier, evaluate result like regression but in some way...   <<--- provide explanation in text !!!!
dont forget class encoding!!!!
'''

import pandas as pd
import numpy as np

import os
import random

from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import Pipeline ### params and model
from sklearn.preprocessing import StandardScaler

# Metrics
from sklearn.metrics import confusion_matrix, fbeta_score, precision_score, recall_score
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay, f1_score, precision_score, recall_score , roc_auc_score
from sklearn.model_selection import GridSearchCV, StratifiedKFold, StratifiedGroupKFold, train_test_split
from sklearn.preprocessing import LabelEncoder

from catboost import CatBoostClassifier, Pool
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns


In [ ]:
df = pd.read_csv('dataset_dota_kills.csv',index_col= 0)#False)

In [ ]:
#df

In [ ]:
df['score_diff'] = df['first_team_score'] - df['second_team_score']
df['rank_diff'] = df['first_team_rating'] - df['second_team_rating']
df['second_team_win_probability'] = 1 - df['first_team_win_probability']

In [ ]:
df['score_diff'].unique()#.sort()

In [ ]:
conditions = [
    (df['score_diff'] <= -31),
    (df['score_diff'] > -31)&(df['score_diff'] < 31),
    (df['score_diff'] > 30) 
    ]

values = [-31, df['score_diff'] , 31]

df['score_diff_'] = np.select(conditions, values)

#df.head()

In [ ]:
##### do class vals encoding for xgboost
conditions = [
    (df['score_diff_'] < 0),
    (df['score_diff_'] >= 0) 
    ]

values = [31 + np.abs(df['score_diff_']), df['score_diff_'] ]

df['score_diff_x'] = np.select(conditions, values)

df.head()

In [ ]:
ax = df['score_diff'].value_counts().plot(kind='bar');#df['score_diff'].value_counts()
ax.bar_label(ax.containers[0])

In [ ]:
ax = df['score_diff_'].value_counts().plot(kind='bar');#df['score_diff'].value_counts()
ax.bar_label(ax.containers[0])

In [ ]:
from pingouin import multivariate_normality
import pandas as pd
import numpy as np
#feats = ['first_team_win_probability' , 'second_team_win_probability' , 'first_team_rating', 'second_team_rating' , 'rank_diff']
data = df[['first_team_win_probability' , 'second_team_win_probability' , 'first_team_rating', 'second_team_rating' , 'rank_diff']]  
# perform the Multivariate Normality Test
multivariate_normality(data, alpha=.05)

In [ ]:
#df['first_team_id'].unique().shape , df['second_team_id'].unique().shape#df['match_id'].unique().shape

In [ ]:
#check separability , make a guess about a nature of the distribution
import matplotlib.pyplot as plt
fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
df_ = df[np.abs(df['score_diff']).isin([30,0,1])]#df_ = df[np.abs(df['score_diff']) == 30] # df_ = df 
ax.scatter(df_['first_team_win_probability'], df_['first_team_rating'], df_['second_team_rating'] , c = df_['score_diff']) 

plt.show()

### Supportive functions

In [ ]:


from math import floor, log10, inf
from scipy import stats

def num_zeros(decimal):
    return 'next' if decimal == 0 else -floor(log10(abs(decimal))) - 1

def multiplier(a):
  for i in a[0].argsort():
    if num_zeros(a[0][i]) != 'next':
      return 10**(num_zeros(a[0][i]) + 1)  
    else:
      continue

def make_array(arr):
  
  m =  multiplier(arr)
  ins = np.around(arr*m).astype(int)[0]
  return np.array([no for no, rep in enumerate(ins) for elem in range(rep)])



def compare_dists(est, est_, XTest):
  ks = []
  for i in range(XTest.shape[0]):
      test_arr = make_array(est_.predict_proba(XTest.iloc[[i]]))
      train_arr = make_array(est.predict_proba(XTest.iloc[[i]]))
      ks.append(stats.kstest(train_arr, test_arr))
  return np.mean([i.pvalue for i in ks])    

### Actual estimation  of distribution and evaluation

In [ ]:
numKFolds = 3 ; randomState = 42 ; seedNum = 512

feats = ['first_team_win_probability' , 'second_team_win_probability' , 'first_team_rating', 'second_team_rating' , 'rank_diff']#feats = ['first_team_win_probability' , 'first_team_rating', 'second_team_rating' , 'rank_diff']
X = df[feats]

y = df['score_diff_']#vY = df_['score_diff_']

models = []
hKFoldSplt = StratifiedKFold(n_splits = numKFolds, shuffle = True, random_state = randomState)
for TrainIdx, TestIdx in hKFoldSplt.split(X, y):

    XTrain, XTest, YTrain, YTest = X.iloc[TrainIdx], X.iloc[TestIdx], y.iloc[TrainIdx], y.iloc[TestIdx]
    
    print(set(YTrain.unique()).symmetric_difference(set(YTest.unique()))) ### <<-- control stratification !!!  #print(XTyrain.unique() ) ;     #print(YTest.unique())
    
    le = LabelEncoder() ;  YTrain = le.fit_transform(YTrain) ;  YTest = le.fit_transform(YTest)
    
    xgbModel =XGBClassifier(n_estimators=50, tree_method="hist", max_depth = 7,  random_state=seedNum, enable_categorical=True) ;  xgbModel_test =XGBClassifier(n_estimators=50, tree_method="hist", max_depth = 7,  random_state=seedNum, enable_categorical=True)
    xgbModel.fit(XTrain, YTrain)
    YPred = xgbModel.predict(XTest)
    xgbModel_test.fit(XTest, YTest)
    
    p = compare_dists(xgbModel, xgbModel_test, XTest)
    print(p)
    #print(accuracy_score(YTest, YPred))
    #print(accuracy_score(YTrain, xgbModel.predict(XTrain)))
    
    
    models.append((xgbModel, xgbModel_test , le , TrainIdx, TestIdx))


### do estimations for actual handicap:

In [ ]:
def handicap_distr(val, est):
    
    vs = est.classes_
    p = 0 
    #for v in  
    return p 

In [ ]:
################ alternative evaluations of result, inspection of approaches

In [ ]:
import matplotlib.pyplot as plt
 
  
def plot_(vals, probs):

    fig = plt.figure(figsize = (10, 5))
    plt.bar(vals, probs, color ='maroon')
    plt.xlabel("vals")
    plt.ylabel("probs")
    plt.show()

In [ ]:
bestid =2
est = models[bestid][0] ; TestIdx = models[bestid][-1] ; le = models[bestid][1]
XTest, YTest = X.iloc[TestIdx], y.iloc[TestIdx]
YTest = le.fit_transform(YTest)
Y_ = le.inverse_transform(YTest)
pred =  le.inverse_transform(est.predict(XTest))
for i in range(10):
    probs = est.predict_proba(XTest.iloc[[0,i]])[1]
    vals = est.classes_
    mu = np.dot(probs,vals)
    std_ = np.dot(np.array(list(map(lambda x : x*x , vals - mu))) , probs)**(1/2)
    print(mu, std_ , YTest[i] , Y_[i] , pred[i])
    plot_(vals, probs)
    #print(est.predict(XTest.iloc[[0,i]])[1] , np.dot(est.predict_proba(XTest.iloc[[0,i]])[1],est.classes_) , YTest[i] )#, est.predict(XTest.iloc[[0,1]])


In [ ]:
'''
bestid =2
est = models[bestid][0] ; TestIdx = models[bestid][-1] ; scaler = models[bestid][1]
XTest, YTest = X.iloc[TestIdx], y.iloc[TestIdx]

XTest = scaler.transform(XTest)
pred =  est.predict(XTest)

for i in range(10):
    probs = est.predict_proba(XTest[i].reshape(1, -1))[0]#probs = est.predict_proba(XTest.iloc[[0,i]])[1]
    vals = est.classes_
    mu = np.dot(probs,vals)
    std_ = np.dot(np.array(list(map(lambda x : x*x , vals - mu))) , probs)**(1/2)
    print(mu, std_ , YTest.iloc[i] ,  pred[i])
    plot_(vals, probs)
    #print(est.predict(XTest.iloc[[0,i]])[1] , np.dot(est.predict_proba(XTest.iloc[[0,i]])[1],est.classes_) , YTest[i] )#, est.predict(XTest.iloc[[0,1]])
'''


In [ ]:
df